In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import re
import os
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

https://ospi.k12.wa.us/data-reporting/data-portal?title=enrollment&field_years1_target_id=All&field_data_domain_target_id=1647&field_level_of_aggregation_target_id=All&field_grade_span_target_id=All

In [ ]:
all_zips = pd.read_excel('Archive/all_zips.xlsx')
parish_zip = pd.read_excel('parish_tracts_zips.xlsx')

parish_zip = parish_zip.merge(all_zips[['school_id', 'public_school_id']], on='school_id', how='left')


In [ ]:
parish_zip.head()

In [ ]:
# Read in Data

# School performance data 
report_card_df1 = pd.read_csv('Public Data/Report_Card_Spring_Assessment_Data_from_2014-15_to_2021-22_20250401.csv', dtype={'CurrentSchoolType': str} )
report_card_df2 = pd.read_csv('Public Data/Report_Card_Assessment_Data_2022-23_School_Year_20250401.csv', dtype={'CurrentSchoolType': str})

# School enrollment data
enrollment_df = pd.read_csv('Public Data/Report_Card_Enrollment_from_2014-15_to_Current_Year_20250401.csv')


# School assessments

In [ ]:
# Filter for school level data
orgs_to_drop = ['district', 'state', 'esd']
school_report_card_df1 = report_card_df1[~report_card_df1['OrganizationLevel'].str.lower().isin(orgs_to_drop)]

report_card_df2['OrganizationLevel'] = report_card_df2['OrganizationLevel'].astype(str).str.strip()
school_report_card_df2 = report_card_df2[report_card_df2['OrganizationLevel'] == 'School']

# Filter for correct years

years_to_drop = ['2014-15', '2015-16', '2016-17', '2017-18']

school_report_card_df1 = school_report_card_df1[~school_report_card_df1['SchoolYear'].isin(years_to_drop)]

# Filter for public schools 
school_report_card_df1 = school_report_card_df1[school_report_card_df1['CurrentSchoolType']=="P"]
school_report_card_df2 = school_report_card_df2[school_report_card_df2['CurrentSchoolType']=="P"]

# Filter for student group: All
school_report_card_df1 = school_report_card_df1[school_report_card_df1['StudentGroupType'] == 'All']
school_report_card_df2 = school_report_card_df2[school_report_card_df2['StudentGroupType'] == 'All']

# Filter for all grades
school_report_card_df1 = school_report_card_df1[school_report_card_df1['GradeLevel']=="All Grades"]
school_report_card_df2 = school_report_card_df2[school_report_card_df2['GradeLevel']=="All Grades"]

# Filter for SBAC – Smarter Balanced Assessment Consortium (used for standardized testing in English Language Arts and Mathematics)
school_report_card_df1 = school_report_card_df1[school_report_card_df1['TestAdministration']=='SBAC']
school_report_card_df2 = school_report_card_df2[school_report_card_df2['TestAdministration']=='SBAC']

# Rename to match 
school_report_card_df2 = school_report_card_df2.rename(columns={'Percent Consistent Grade Level Knowledge And Above':'PercentMetStandard'})




In [ ]:
school_report_card_df1.columns

In [ ]:
# Drop unwanted columns 

filtered_school_report_df1 = school_report_card_df1.drop(['OrganizationLevel','County', 'ESDName',
       'ESDOrganizationID', 'DistrictCode', 'DistrictName','DistrictOrganizationId', 'SchoolOrganizationid', 'TestAdministration', 
       'Suppression', 'Count of Students Expected to Test','CountofStudentsExpectedtoTest(including previously passed)',
       'PercentMetTestedOnly', 'Percent No Score', 'DataAsOf', 'GradeLevel', 'CountMetStandard', 'CurrentSchoolType',
       'StudentGroupType', 'StudentGroup'], axis=1)

filtered_school_report_df2 = school_report_card_df2.drop(['OrganizationLevel','County', 'ESDName',
       'ESDOrganizationId', 'DistrictCode', 'DistrictName',
       'DistrictOrganizationId', 'TestAdministration', 'DAT', 'Count of Students Expected to Test',
       'Count of Students Expected to Test (included previously passed)',
       'Count Consistent Grade Level Knowledge And Above', 'Percent Consistent Tested Only', 'PercentNoScore',
       'PercentParticipation', 'DataAsOf', 'GradeLevel', 'SchoolOrganizationId', 'CurrentSchoolType',
       'StudentGroupType', 'StudentGroup'], axis=1)

In [ ]:
# Check columns
filtered_school_report_df1.columns

In [ ]:
# Check columns
filtered_school_report_df2.columns

In [ ]:
combined_school_report = pd.concat([filtered_school_report_df1, filtered_school_report_df2], ignore_index=True)

# Change school code into int
combined_school_report['SchoolCode'] = combined_school_report['SchoolCode'].astype(int)

# Change percent into float
combined_school_report['PercentMetStandard'] = combined_school_report['PercentMetStandard'].str.replace(r'[^0-9.]', '', regex=True)
combined_school_report['PercentMetStandard'] = combined_school_report['PercentMetStandard'].replace('', np.nan).astype(float)/100

# Drop homeschool - no enrollment data
combined_school_report = combined_school_report[combined_school_report['SchoolCode'] != 5390]

In [ ]:
parish_zip.columns

In [ ]:
parish_zip['public_school_id'] = parish_zip['public_school_id'].astype(str)

# Split the codes into lists
parish_zip['public_school_id'] = parish_zip['public_school_id'].str.split(',')

# Remove any extra whitespace
parish_zip['public_school_id'] = parish_zip['public_school_id'].apply(lambda x: [code.strip() for code in x])

# Explode the list into separate rows
df_expanded = parish_zip.explode('public_school_id').reset_index(drop=True)


In [ ]:
df_expanded

In [ ]:
combined_school_report['SchoolCode'] = combined_school_report['SchoolCode'].astype(str)

school_codes_zips = df_expanded.dropna(subset=['public_school_id'])

school_codes_zips['public_school_id'] = school_codes_zips['public_school_id'].astype(str)

parish_codes = school_codes_zips[['school_id', 'public_school_id']]

parish_codes['school_id'] = parish_codes['school_id'].astype(str)


In [ ]:
combined_school_report

In [ ]:
school_codes_zips

In [ ]:
print(parish_codes[parish_codes['public_school_id'] == '3572'])

In [ ]:
# merge for zip codes
combined_school_report = combined_school_report.merge(parish_codes, left_on='SchoolCode', right_on='public_school_id', how='left')

In [ ]:
combined_school_report[combined_school_report['SchoolCode']=='3572']

In [ ]:
combined_school_report.drop(columns='public_school_id', inplace=True)

In [ ]:
combined_school_report

# Enrollments

In [ ]:
enrollment_df.columns

In [ ]:
# Filter for school level
school_enrollment = enrollment_df[enrollment_df['OrganizationLevel'] == 'School']

# Remove years 
years_to_drop = ['2014-15', '2015-16', '2016-17', '2017-18', '2023-24', '2024-25']
school_enrollment = school_enrollment[~school_enrollment['SchoolYear'].isin(years_to_drop)]

# Use all grades
allgrade_enrollment = school_enrollment[school_enrollment['Gradelevel']=='All Grades']

# Public school only
public_enrollment_df = allgrade_enrollment[allgrade_enrollment['CurrentSchoolType'] == 'P']

# Change school code to int 
public_enrollment_df.loc[:, 'SchoolCode'] = public_enrollment_df['SchoolCode'].astype(int)



In [ ]:
# Drop unused columns
filtered_enrollment_df= public_enrollment_df.drop(['OrganizationLevel', 'County', 'ESDName',
       'ESDOrganizationID', 'DistrictCode', 'DistrictName',
       'DistrictOrganizationId','SchoolOrganizationid', 'CurrentSchoolType','English Language Learners', 'Highly Capable', 'Homeless', 'Low-Income',
       'Migrant', 'Military Parent', 'Mobile', 'Section 504',
       'Students with Disabilities', 'Non-English Language Learners',
       'Non-Highly Capable', 'Non-Homeless', 'Non-Low Income', 'Non Migrant',
       'Non Military Parent', 'Non Mobile', 'Non Section 504',
       'Students without Disabilities', 'FosterCare', 'Non-FosterCare',
       'DataAsOf', 'Gender X'], axis=1).reset_index(drop=True)

In [ ]:
filtered_enrollment_df.columns

In [ ]:
filtered_enrollment_df

In [ ]:
filtered_enrollment_df['SchoolCode'] = filtered_enrollment_df['SchoolCode'].astype(int).astype(str)

In [ ]:
filtered_enrollment_df

In [ ]:
parish_codes.head()

In [ ]:
enrollment_report = filtered_enrollment_df.merge(parish_codes, left_on='SchoolCode', right_on='public_school_id', how='left')

In [ ]:
enrollment_report

In [ ]:
enrollment_report.drop(columns='public_school_id', axis=1, inplace=True)

In [ ]:
enrollment_report.head()

#### Aggregate assessments by parish 

In [ ]:
parish_enrollment_report = enrollment_report

In [ ]:
schools_names = parish_enrollment_report['SchoolName'].unique()

In [ ]:
for name in schools_names:
    print(name)

In [ ]:
# Drop high schools and online/virtual schools
schools_to_drop = ['High School', '9-12', 'Intermediate', 'High', 'Online', 'Virtual', 'Industrial Design Engineering and Arts', 'District', 'Technology Access Foundation Academy at Saghalie', 'Impact Public Schools', 'HS', 'Preschool', 'C O Sorenson', 'Summit Public School: Atlas', 
                   'Rainier Valley Leadership Academy', 'Klahowya Secondary', 'Odyssey Multiage Program', 'Washington Connections Academy - Mary M. Knight', 'Tacoma School of the Arts',
                   'University Place Primary', 'Lakeview Hope Academy', 'General William H. Harrison Preparatory School', 'Clover Park Early Learning Program',
                   'Conway School District 317', 'ECEAP', 'Pathfinder Kindergarten Center', 'Maplewood Parent Coop', 'Edwin Pratt Learning Center', 'Bellevue Digital Discovery', 'Central Primary Center']

pattern = '|'.join(schools_to_drop)

parish_school_report = parish_enrollment_report[~parish_enrollment_report['SchoolName'].str.contains(pattern, case=False, na=False)]


In [ ]:
parish_school_report.columns

In [ ]:
combined_school_report.columns

In [ ]:
parish_assessment_agg = combined_school_report.groupby(['SchoolYear', 'TestSubject', 'school_id']).agg(
    PercentMetStandard=('PercentMetStandard', 'mean'),
    PercentLevel1=('PercentLevel1', 'mean'),
    PercentLevel2=('PercentLevel2', 'mean'),
    PercentLevel3=('PercentLevel3', 'mean'),
    PercentLevel4=('PercentLevel4', 'mean')
).reset_index()

# Pivot each metric separately and rename columns
metrics = ['PercentMetStandard', 'PercentLevel1', 'PercentLevel2', 'PercentLevel3', 'PercentLevel4']
df_list = []

for metric in metrics:
    pivoted = parish_assessment_agg.pivot_table(
        index=['SchoolYear', 'school_id'],
        columns='TestSubject',
        values=metric,
        aggfunc='first'
    )
    # Rename columns to include the metric name
    pivoted.columns = [f'{metric}_{col}' for col in pivoted.columns]
    df_list.append(pivoted)

# Combine all the pivoted DataFrames
parish_assessment_agg = pd.concat(df_list, axis=1)

# Reset index if you want SchoolYear and Name as columns rather than index
parish_assessment_agg = parish_assessment_agg.reset_index()

In [ ]:
parish_assessment_agg

#### Aggregate enrollment by parish 

In [ ]:
parish_enrollment_report.columns

In [ ]:

# Drop high schools and online/virtual schools
schools_to_drop = ['High School', '9-12', 'Intermediate', 'High', 'Online', 'Virtual', 'Industrial Design Engineering and Arts', 'District', 'Technology Access Foundation Academy at Saghalie', 'Impact Public Schools', 'HS', 'Preschool', 'C O Sorenson', 'Summit Public School: Atlas', 
                   'Rainier Valley Leadership Academy', 'Klahowya Secondary', 'Odyssey Multiage Program', 'Washington Connections Academy - Mary M. Knight', 'Tacoma School of the Arts',
                   'University Place Primary', 'Lakeview Hope Academy', 'General William H. Harrison Preparatory School', 'Clover Park Early Learning Program',
                   'Conway School District 317', 'ECEAP', 'Pathfinder Kindergarten Center', 'Maplewood Parent Coop', 'Edwin Pratt Learning Center', 'Bellevue Digital Discovery', 'Central Primary Center',
                   'Crescent School', 'Meadow Crest Early Childhood Education Center', ]
pattern = '|'.join(schools_to_drop)

parish_school_enrollment = parish_enrollment_report[~parish_enrollment_report['SchoolName'].str.contains(pattern, case=False, na=False)]


In [ ]:
parish_school_enrollment['SchoolName'].unique()

In [ ]:
school_codes = combined_school_report['SchoolCode'].unique()

In [ ]:
filtered_parish_school_enrollment = parish_school_enrollment.copy()
#filtered_parish_school_enrollment = parish_school_enrollment[parish_school_enrollment['SchoolCode'].isin(school_codes)]

In [ ]:
filtered_parish_school_enrollment.columns

In [ ]:
parish_enrollment_agg = filtered_parish_school_enrollment.groupby(['SchoolYear', 'school_id']).agg(
    all_students=('All Students', 'sum'),
    female=('Female', 'sum'),
    male=('Male', 'sum'),
    americanindian_alaskanative=('American Indian/ Alaskan Native', 'sum'),
    asian=('Asian', 'sum'),
    black_africanamerican=('Black/ African American', 'sum'),
    hispanic_latino=('Hispanic/ Latino of any race(s)', 'sum'),
    nativehawaiian_pacificislander=('Native Hawaiian/ Other Pacific Islander', 'sum'),
    two_more=('Two or More Races', 'sum'),
    white=('White', 'sum'),
    school_count=('SchoolName', 'nunique')
).reset_index()

In [ ]:
parish_assessment_agg

In [ ]:
combined_report_enrollment=parish_enrollment_agg.merge(parish_assessment_agg, on=['SchoolYear', 'school_id'], how='left')
merged_df = combined_report_enrollment

In [ ]:
merged_df

In [ ]:
merged_df.columns

In [ ]:
# Change year 
merged_df['SchoolYear'] = merged_df['SchoolYear'].astype(str).replace({
    '2018-19': '2018_19',
    '2019-20': '2019_20',
    '2020-21': '2020_21',
    '2021-22': '2021_22',
    '2022-23': '2022_23'
})

In [ ]:
merged_df

In [ ]:
merged_df.isna().sum()

In [ ]:
# Numeric and Non-Numeric
numeric_df = merged_df.select_dtypes(include=[np.number])
non_numeric_df = merged_df.select_dtypes(exclude=[np.number])

# Initialize MICE-style imputer
imputer = IterativeImputer(random_state=0)

# Fit and transform the data
imputed_data = imputer.fit_transform(numeric_df)

imputed_df = pd.DataFrame(imputed_data, 
                          columns=numeric_df.columns,
                          index=merged_df.index)

# Concatenate 
imputed_merged = pd.concat([non_numeric_df, imputed_df], axis=1)


In [ ]:
imputed_merged.columns

In [ ]:
imputed_merged.rename(columns={'americanindian_alaskanative' : 'pub_americanindian_alaskanative', 
                               'asian' : 'pub_asian', 
                               'black_africanamerican': 'pub_black_africanamerican',
                               'hispanic_latino' : 'pub_hispanic_latino', 
                               'nativehawaiian_pacificislander' : 'pub_nativehawaiian_pacificislander', 
                               'two_more' : 'pub_two_more',
                               'white' : 'pub_white',
                               'female' : 'pub_female',
                               'male': 'pub_male',
                               'all_students': 'pub_all_students'}, inplace=True)

In [ ]:
imputed_merged

In [ ]:
imputed_merged.to_csv('parish_nces_imputed_data.csv')